<a href="https://colab.research.google.com/github/r-hugoalves/EDA_com_SQL_e_AWS/blob/master/Explora%C3%A7%C3%A3o_e_an%C3%A1lise_de_dados_de_cr%C3%A9dito_com_SQL_na_AWS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploração e análise de dados de crédito com SQL na AWS

Nesse estudo, vamos analisar um conjunto de dados que representam informações de clientes de um banco.

Para o nosso estudo, vamos utilizar as ferramentas S3 e Athena da AWS e uma parte do conjunto de dados disponibilizados [nesse](https://github.com/andre-marcos-perez/ebac-course-utils/tree/main/dataset) repositório*. Além disso, todos os gráficos aqui apresentados foram gerados utilizando o excel.

**.: não vamos trabalhar com todo o conjunto de dados para não ultrapassar o período gratuito da AWS. Por isso, utilizaremos apenas as 2563 primeiras linhas.*

## Dados

Legenda dos dados presentes na tabela:

* idade = idade do cliente
* sexo = sexo do cliente (F ou M)
* dependentes = número de dependentes do cliente
* escolaridade = nível de escolaridade do clientes
* salario_anual = faixa salarial do cliente
* tipo_cartao = tipo de cartao do cliente
* qtd_produtos = quantidade de produtos comprados nos últimos 12 meses
* iteracoes_12m = quantidade de iterações/transacoes nos ultimos 12 meses
* meses_inativo_12m = quantidade de meses que o cliente ficou inativo
* limite_credito = limite de credito do cliente
* valor_transacoes_12m = valor das transações dos ultimos 12 meses
* qtd_transacoes_12m = quantidade de transacoes dos ultimos 12 meses

## Extração dos dados

## Exploração dos dados

## Análise dos dados

## Conclusão